Creating Reviewer-Recipe-Rating matrix :

In [11]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
final_processed_dataset=pd.read_csv(r"Final_Processed_Dataset.csv")

reviewer_recipe_rating_matrix = final_processed_dataset.pivot_table(index='ReviewerName', columns='Name', values='Rating')
reviewer_recipe_rating_relation_matrix=reviewer_recipe_rating_matrix.fillna(0)

Calculating cosine similarity matrix :

In [13]:
cosine_similarity_matrix=cosine_similarity(reviewer_recipe_rating_relation_matrix)

cosine_similarity_matrix = pd.DataFrame(cosine_similarity_matrix, index=reviewer_recipe_rating_relation_matrix.index, columns=reviewer_recipe_rating_relation_matrix.index)

ReviewerName     .. Ameera ..    Carly    Pamela    Poppy   - Carla -  \
ReviewerName                                                            
 .. Ameera ..              1.0      0.0  0.000000      0.0    0.00000   
 Carly                     0.0      1.0  0.000000      0.0    0.00000   
 Pamela                    0.0      0.0  1.000000      0.0    0.02572   
 Poppy                     0.0      0.0  0.000000      1.0    0.00000   
- Carla -                  0.0      0.0  0.025720      0.0    1.00000   
...                        ...      ...       ...      ...        ...   
znglass                    0.0      0.0  0.021809      0.0    0.00000   
zoe.s.true                 0.0      0.0  0.000000      0.0    0.00000   
zombiesgirl                0.0      0.0  0.000000      0.0    0.00000   
zoo66                      0.0      0.0  0.000000      0.0    0.00000   
zoyie                      0.0      0.0  0.000000      0.0    0.00000   

ReviewerName     -------  -JL-6680     -JoeB  -Oce

c:\Users\nishi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:208: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


Predicting Ratings for all recipes where rating is nan for a each user one by one :

In [ ]:
reviewer_recipe_rating_matrix